<a href="https://colab.research.google.com/github/AmitKPandey11/AI-agent-project/blob/main/Email_Agent_CrewAI_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Email Agent with CrewAI — Colab Friendly

This notebook installs CrewAI (if available in the runtime), sets up a small crew that integrates:
- Web search tool (SerpAPI primary, DuckDuckGo fallback)
- RAG retriever (Sentence-Transformers + FAISS, TF-IDF fallback)
- Sentiment analyzer (HuggingFace pipeline with heuristic fallback)
- Email composer agent that aggregates outputs into a reply

The notebook detects install issues and falls back to a local synchronous pipeline so the demo always runs on Colab.

**Run all cells sequentially.**

In [1]:
# Step 0: environment info
import sys, os
print('Python', sys.version)
try:
    import torch
    print('Torch available. CUDA:', torch.cuda.is_available())
except Exception as e:
    print('Torch not available or import failed:', e)
os.environ.setdefault('HF_HOME', '/content/hf_cache')
os.environ.setdefault('TRANSFORMERS_OFFLINE', '0')


Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch available. CUDA: True


'0'

In [2]:
# Step 1: Install CrewAI + dependencies (may take 60-120s). Notebook will continue even if crewai install fails.
import subprocess, sys
pkgs = ['crewai', 'sentence-transformers', 'faiss-cpu', 'transformers', 'scikit-learn', 'requests', 'python-dotenv']
print('Installing:', pkgs)
try:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)
    print('Install completed')
except Exception as e:
    print('Some installs failed (this is okay). Error:', e)
    print('Proceeding with fallback pipeline if crewai is unavailable')


Installing: ['crewai', 'sentence-transformers', 'faiss-cpu', 'transformers', 'scikit-learn', 'requests', 'python-dotenv']
Install completed


## Step 2: Prepare sample documents (knowledge base)

In [3]:
from pathlib import Path, PurePath
import json
KB_DIR = Path('email_agent_knowledge')
KB_DIR.mkdir(exist_ok=True)
docs = {
    'product_policy.txt': 'Our product refund policy allows customers to request refunds within 30 days of purchase if the product is unused and in its original packaging.',
    'campaign_report.txt': 'The July campaign saw a 20 percent drop in conversions compared to June. The budget pacing indicated overspend on low-performing channels.',
    'safety_guidelines.txt': 'For escalations, contact the on-call manager. Always include ticket ID and detailed reproduction steps when reporting safety incidents.'
}
for fname, txt in docs.items():
    p = KB_DIR / fname
    p.write_text(txt, encoding='utf-8')
print('Wrote sample docs to', KB_DIR)
json.dump([{ 'source': str(KB_DIR / k), 'text': v } for k, v in docs.items()], open(KB_DIR / 'meta.json','w'), indent=2)


Wrote sample docs to email_agent_knowledge


## Step 3: RAG builder (Sentence-Transformers + FAISS, TF-IDF fallback)

In [4]:
import numpy as np
from pathlib import Path
KB_DIR = Path('email_agent_knowledge')
emb_path = KB_DIR / 'embeddings.npy'
use_st = False
try:
    from sentence_transformers import SentenceTransformer
    st_model = SentenceTransformer('all-MiniLM-L6-v2')
    docs = [ (KB_DIR / f).read_text(encoding='utf-8') for f in ['product_policy.txt','campaign_report.txt','safety_guidelines.txt'] ]
    emb = st_model.encode(docs, show_progress_bar=False)
    np.save(emb_path, emb)
    use_st = True
    print('Built ST embeddings', emb.shape)
except Exception as e:
    print('SentenceTransformer failed or offline, falling back to TF-IDF. Error:', e)
    from sklearn.feature_extraction.text import TfidfVectorizer
    docs = [ (KB_DIR / f).read_text(encoding='utf-8') for f in ['product_policy.txt','campaign_report.txt','safety_guidelines.txt'] ]
    vec = TfidfVectorizer()
    emb = vec.fit_transform(docs).toarray().astype('float32')
    np.save(emb_path, emb)
    # save vectorizer for retrieval fallback
    import pickle
    pickle.dump(vec, open(KB_DIR / 'tfidf_vec.pkl','wb'))
    use_st = False
    print('Built TF-IDF embeddings', emb.shape)

# Build FAISS index if possible
try:
    import faiss
    emb = np.load(emb_path)
    # normalize for cosine
    norms = np.linalg.norm(emb, axis=1, keepdims=True) + 1e-12
    embn = emb / norms
    d = embn.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(embn.astype('float32'))
    faiss.write_index(index, str(KB_DIR / 'faiss.index'))
    has_faiss = True
    print('FAISS index written, dim=', d)
except Exception as e:
    print('FAISS unavailable or index build failed:', e)
    has_faiss = False


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Built ST embeddings (3, 384)
FAISS index written, dim= 384


## Step 4: Define tools (web search + sentiment + rag retrieve)

In [5]:
import requests, json
from sklearn.metrics.pairwise import cosine_similarity
KB_DIR = Path('email_agent_knowledge')

class WebSearchTool:
    def __init__(self, serpapi_key=None):
        self.serpapi_key = serpapi_key
    def search_serpapi(self, query, num=3):
        if not self.serpapi_key:
            raise ValueError('No SERPAPI key')
        url = 'https://serpapi.com/search'
        params = {'q': query, 'api_key': self.serpapi_key, 'engine': 'google', 'num': num}
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        res = []
        for it in data.get('organic_results', [])[:num]:
            res.append({'title': it.get('title'), 'snippet': it.get('snippet'), 'link': it.get('link')})
        return res
    def search(self, query, num=3):
        try:
            return self.search_serpapi(query, num)
        except Exception:
            try:
                r = requests.get('https://api.duckduckgo.com', params={'q': query, 'format': 'json'}, timeout=8)
                j = r.json()
                return [{'title': 'DuckDuckGo', 'snippet': j.get('AbstractText',''), 'link': ''}]
            except Exception:
                return []

def analyze_sentiment(text):
    try:
        from transformers import pipeline
        sa = pipeline('sentiment-analysis')
        return sa(text[:512])[0]
    except Exception:
        low = text.lower()
        if any(w in low for w in ['angry','upset','issue','problem','disappointed']):
            return {'label':'NEGATIVE','score':0.9}
        return {'label':'POSITIVE','score':0.8}

def rag_retrieve(query, k=3):
    # Prefer FAISS + ST if available
    meta = json.load(open(KB_DIR / 'meta.json'))
    try:
        import faiss
        emb = np.load(KB_DIR / 'embeddings.npy')
        qvec = None
        if use_st:
            qvec = st_model.encode([query]); qvec = qvec / (np.linalg.norm(qvec, axis=1, keepdims=True)+1e-12)
        else:
            # TF-IDF query
            import pickle
            vec = pickle.load(open(KB_DIR / 'tfidf_vec.pkl','rb'))
            qvec = vec.transform([query]).toarray().astype('float32')
            qvec = qvec / (np.linalg.norm(qvec, axis=1, keepdims=True)+1e-12)
        index = faiss.read_index(str(KB_DIR / 'faiss.index'))
        D,I = index.search(qvec.astype('float32'), k)
        out = []
        for idx, score in zip(I[0], D[0]):
            out.append((meta[int(idx)], float(score)))
        return out
    except Exception as e:
        # fallback cosine on saved embeddings
        emb = np.load(KB_DIR / 'embeddings.npy')
        from sklearn.feature_extraction.text import TfidfVectorizer
        try:
            vec = __import__('pickle').load(open(KB_DIR / 'tfidf_vec.pkl','rb'))
            qv = vec.transform([query]).toarray().astype('float32')
            sims = cosine_similarity(qv, emb)[0]
        except Exception:
            # As last resort, naive token overlap
            docs = [d['text'] for d in meta]
            sims = [sum(1 for t in query.split() if t in d.split()) for d in docs]
        idxs = sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)[:k]
        meta = json.load(open(KB_DIR / 'meta.json'))
        return [(meta[i], float(sims[i])) for i in idxs]


## Step 5: Try to use CrewAI to wire agents; if unavailable, run a local pipeline

In [6]:
CREW_AVAILABLE = False
try:
    import crewai
    from crewai import Agent, Crew
    CREW_AVAILABLE = True
    print('CrewAI imported successfully')
except Exception as e:
    print('CrewAI not available or import failed:', e)

webtool = WebSearchTool(os.getenv('SERPAPI_API_KEY'))
def run_local_pipeline(email_text, subject=None):
    # 1) sentiment
    sent = analyze_sentiment(email_text)
    # 2) extract keywords (tiny heuristic)
    words = [w.strip('.,?!') for w in email_text.split()]
    words = [w for w in words if len(w)>4]
    freq = {}
    for w in words:
        freq[w.lower()] = freq.get(w.lower(),0)+1
    keywords = sorted(freq, key=freq.get, reverse=True)[:3]
    # 3) web search
    web_results = []
    if keywords:
        try:
            web_results = webtool.search(' '.join(keywords))
        except Exception:
            web_results = []
    # 4) rag retrieve
    docs = rag_retrieve(email_text, k=3)
    # 5) compose
    tone = 'empathetic' if str(sent.get('label','')).lower().startswith('neg') else 'neutral'
    subj = subject or 'Customer inquiry'
    body = f'Subject: Re: {subj}\nTone: {tone}\n\nKey findings:\n'
    for d,s in docs:
        body += f"- {d['text'][:200]} (score {round(s,3)})\n"
    if web_results:
        body += '\nWeb findings:\n'
        for r in web_results[:3]:
            body += f"- {r.get('title')} - {r.get('snippet')}\n"
    body += '\nNext steps: We will investigate and follow up within 2 business days.\n\nBest,\nSupport Team'
    return {'subject': 'Re: ' + subj, 'body': body, 'sentiment': sent, 'keywords': keywords}

if CREW_AVAILABLE:
    print('You can construct a CrewAI crew here programmatically using Agent and Crew objects.')
else:
    print('Running local pipeline demo. Use run_local_pipeline(email_text) to test.')


CrewAI imported successfully
You can construct a CrewAI crew here programmatically using Agent and Crew objects.


## Step 6: Demo run (local pipeline)

In [7]:
test_email = "Hi team, I've noticed our July campaign overspent and conversion dropped by 20%. Can you explain what happened and how we'll fix it? Thanks, Rahul"
out = run_local_pipeline(test_email, subject='July campaign overspend')
print('--- GENERATED EMAIL ---')
print('Subject:', out['subject'])
print('\nBody:\n', out['body'])
print('\nSentiment:', out['sentiment'])
print('\nKeywords:', out['keywords'])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


--- GENERATED EMAIL ---
Subject: Re: July campaign overspend

Body:
 Subject: Re: July campaign overspend
Tone: empathetic

Key findings:
- The July campaign saw a 20 percent drop in conversions compared to June. The budget pacing indicated overspend on low-performing channels. (score 0.712)
- Our product refund policy allows customers to request refunds within 30 days of purchase if the product is unused and in its original packaging. (score 0.07)
- For escalations, contact the on-call manager. Always include ticket ID and detailed reproduction steps when reporting safety incidents. (score 0.001)

Web findings:
- DuckDuckGo - 

Next steps: We will investigate and follow up within 2 business days.

Best,
Support Team

Sentiment: {'label': 'NEGATIVE', 'score': 0.9988725781440735}

Keywords: ['noticed', 'campaign', 'overspent']


## Step 7: Packaging note
If you want me to package this notebook + `email_agent_knowledge/` into a ZIP for download, say 'package and share' and I'll produce the ZIP here for you to download.